## Code to find intersections of ICESat-2 and ATM data

**by Allison Chartrand**

**June 2019 ICESat-2 Hackweek**


In [2]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
from scipy.spatial import distance

Code to extract single tracks of IceSat-2 data

In [67]:
ATL06data = pd.read_csv('../extractData/IceSat2ZachData')

In [62]:
ATL06data['track'].unique()

array(['gt3r', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l'], dtype=object)

In [63]:
ATL06data.date.unique()

array(['2019-01-04 12:24:25', '2019-02-15 10:09:55',
       '2018-10-29 05:05:08', '2019-01-16 01:09:49',
       '2018-11-23 03:49:31', '2018-12-10 02:50:30',
       '2018-12-19 12:57:40', '2019-01-08 01:26:37',
       '2018-11-20 14:21:44', '2018-12-26 02:17:13',
       '2019-02-17 23:37:46', '2018-11-16 14:30:09',
       '2018-11-27 03:41:06', '2018-12-27 12:41:01',
       '2018-10-21 05:21:45', '2018-12-23 12:49:22',
       '2018-10-22 15:45:34', '2018-12-14 02:42:09',
       '2018-12-15 13:05:59', '2019-01-20 01:01:33',
       '2018-12-18 02:33:52', '2019-01-13 11:42:01',
       '2019-01-29 11:08:55', '2018-11-15 04:06:22',
       '2018-11-19 03:57:57', '2018-12-02 13:56:37',
       '2018-10-17 05:30:05', '2019-01-17 11:33:37',
       '2018-10-25 05:13:27', '2018-10-30 15:28:58',
       '2019-01-21 11:25:25', '2019-02-13 23:46:06',
       '2018-10-26 15:37:16', '2019-01-28 00:45:05',
       '2019-01-12 01:18:14', '2018-12-30 02:08:55',
       '2019-01-25 11:17:11', '2018-12-31 12:3

Define dataframe for output

In [68]:
Intersections = {'Flow_dist_ATM':[],'z_ATM':[],'idx_ATM':[],'z_ATL06':[],'t_ATL06':[],'idx_ATL06':[]}
Intersections = pd.DataFrame(data = Intersections)

Intersections.head()

,Flow_dist_ATM,z_ATM,idx_ATM,z_ATL06,t_ATL06,idx_ATL06


Define nearest neighbor algorithm

In [69]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    closest_dist = np.min(distance.cdist([node], nodes,'euclidean'))
    return closest_index, closest_dist

Get target points from ATM csv and put into dataframe

In [70]:
ATMdata = pd.read_csv('~/xtrak/extractData/atmZach04_18_19.csv')

tpoints = ATMdata[['PS_x','PS_y']].copy()
tpoints = tpoints.values


Get query points from ATL06 csv and put into dataframe

In [76]:
i = 0
for day in ATL06data.date.unique()[0:4]:
    for tr in ATL06data.track.unique():
        #dfTran is a single radar transect
        dfTran = ATL06data.query('date == @day & track == @tr')
        if dfTran.shape[0] == 0:
            continue
        dfTran.head()    
        qpoints = dfTran[['x','y']].copy()
        qpoints = qpoints.values
        close_idx = []
        min_dist = []
        for j in range(len(tpoints)):
            close_idxT,min_distT = closest_node(tpoints[i,:], qpoints)
            close_idx = np.append(close_idx,[close_idxT])
            min_dist = np.append(min_dist,[min_distT])

        tpt_idx = np.argmin(min_dist)
        qpt_idx = int(close_idx[tpt_idx])
        AddDatarow = {'Flow_dist_ATM':[ATMdata.loc[tpt_idx,'dist_along']],
                      'z_ATM':[ATMdata.loc[tpt_idx,'ATM_elev']],
                      'idx_ATM':[tpt_idx],
                      'z_ATL06':[ATL06data.loc[qpt_idx,'h']],
                      't_ATL06':[ATL06data.loc[qpt_idx,'date']],
                      'idx_ATL06':[qpt_idx]}
        AddDatarow = pd.DataFrame(data = AddDatarow)
        Intersections.append(AddDatarow)
        print(AddDatarow)

In [77]:
Intersections.head()

,Flow_dist_ATM,z_ATM,idx_ATM,z_ATL06,t_ATL06,idx_ATL06


In [52]:



Intersections.to_csv(index=False)